In [53]:
import os
import numpy as np
import pandas as pd
import librosa
# Replace this with your actual path
preprocessed_dataset_path = r"..\PreProcessedDataSet_for_ML1"

n_fft = 400 # 25 ms window
hop_length = 160 # 10 ms step → 60% overlap


In [54]:
def extract_features_from_audio(file_path, sr=16000):
    try:
        y, _ = librosa.load(file_path, sr=sr)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, n_fft=n_fft, hop_length=hop_length)
        mfcc_mean = mfcc.mean(axis=1)
        mfcc_std = mfcc.std(axis=1)
        zcr_mean = librosa.feature.zero_crossing_rate(y, frame_length=n_fft, hop_length=hop_length).mean()
        rms_mean = librosa.feature.rms(y=y, frame_length=n_fft, hop_length=hop_length).mean()
        centroid_mean = librosa.feature.spectral_centroid(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).mean()
        bandwidth_mean = librosa.feature.spectral_bandwidth(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).mean()
        rolloff_mean = librosa.feature.spectral_rolloff(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).mean()
        chroma_mean = librosa.feature.chroma_stft(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).mean(axis=1)

        return np.concatenate((mfcc_mean, mfcc_std, [zcr_mean, rms_mean, centroid_mean, bandwidth_mean, rolloff_mean], chroma_mean))
    except Exception as e:
        print(f"Error with {file_path}: {e}")
        return None


In [55]:
features = []
filenames = []

for file in os.listdir(preprocessed_dataset_path):
    if file.lower().endswith(".wav"):
        path = os.path.join(preprocessed_dataset_path, file)
        result = extract_features_from_audio(path)
        if result is not None:
            print(f"features are extracted from : {file}")
            features.append(result)
            filenames.append(file)

mfcc_cols = [f"mfcc{i+1}_mean" for i in range(13)] + [f"mfcc{i+1}_std" for i in range(13)]
spec_cols = ["zcr_mean", "rms_mean", "centroid_mean", "bandwidth_mean", "rolloff_mean"]
chroma_cols = [f"chroma{i+1}_mean" for i in range(12)]
columns = ["file name"] + mfcc_cols + spec_cols + chroma_cols

df = pd.DataFrame(features, columns=mfcc_cols + spec_cols + chroma_cols)
df.insert(0, "file_name", filenames)
df.to_csv(r"Analytics\audio_features.csv", index=False)


features are extracted from : 1001_DFA_ANG_XX.wav
features are extracted from : 1001_DFA_DIS_XX.wav
features are extracted from : 1001_DFA_FEA_XX.wav
features are extracted from : 1001_DFA_HAP_XX.wav
features are extracted from : 1001_DFA_NEU_XX.wav
features are extracted from : 1001_DFA_SAD_XX.wav
features are extracted from : 1001_IEO_ANG_HI.wav
features are extracted from : 1001_IEO_ANG_LO.wav
features are extracted from : 1001_IEO_ANG_MD.wav
features are extracted from : 1001_IEO_DIS_HI.wav
features are extracted from : 1001_IEO_DIS_LO.wav
features are extracted from : 1001_IEO_DIS_MD.wav
features are extracted from : 1001_IEO_FEA_HI.wav
features are extracted from : 1001_IEO_FEA_LO.wav
features are extracted from : 1001_IEO_FEA_MD.wav
features are extracted from : 1001_IEO_HAP_HI.wav
features are extracted from : 1001_IEO_HAP_LO.wav
features are extracted from : 1001_IEO_HAP_MD.wav
features are extracted from : 1001_IEO_NEU_XX.wav
features are extracted from : 1001_IEO_SAD_HI.wav


In [56]:
def extract_features_from_audio2(file_path, sr=16000):
    try:
        y, _ = librosa.load(file_path, sr=sr)

        result = np.array([])

        # MFCC: 13 coefficients (mean & std)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, n_fft=n_fft, hop_length=hop_length)
        mfcc_mean = np.mean(mfcc.T, axis=0)
        mfcc_std = np.std(mfcc.T, axis=0)
        result = np.hstack((result, mfcc_mean, mfcc_std))

        # ZCR
        zcr = np.mean(librosa.feature.zero_crossing_rate(y=y, frame_length=n_fft, hop_length=hop_length).T, axis=0)
        result = np.hstack((result, zcr))

        # RMS
        rms = np.mean(librosa.feature.rms(y=y, frame_length=n_fft, hop_length=hop_length).T, axis=0)
        result = np.hstack((result, rms))

        # Spectral Centroid
        centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).T, axis=0)
        result = np.hstack((result, centroid))

        # Spectral Bandwidth
        bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).T, axis=0)
        result = np.hstack((result, bandwidth))

        # Spectral Rolloff
        rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).T, axis=0)
        result = np.hstack((result, rolloff))

        # Chroma STFT
        chroma_mean = np.mean(librosa.feature.chroma_stft(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).T, axis=0)
        result = np.hstack((result, chroma_mean))

        return result

    except Exception as e:
        print(f"Error with {file_path}: {e}")
        return None
